# Chapter 11 - Deep Network on CIFAR10 Dataset

In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd 
import numpy as np 
import ssl
import matplotlib.pyplot as plt 
from functools import partial 
import os

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.cifar10.load_data()

In [4]:
X_train, y_train, X_valid, y_valid = X_train_full[:40000], y_train_full[:40000], X_train_full[40000:], y_train_full[40000:]

In [5]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

DenseLayer = partial(
    keras.layers.Dense, 
    activation = 'elu',
    kernel_initializer = 'he_normal'
)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[32, 32, 3]))
for _ in range(20):
    model.add(DenseLayer(100))

In [6]:
model.add(keras.layers.Dense(10, activation='softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 100)               307300    
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 100)               10100     
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 100)               1

In [16]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint('models/ch11_model.h5', save_best_only=True)
run_index = 1
run_logdir = os.path.join(os.curdir, 'cifar10_logs', 'run_{:03d}'.format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
callbacks = [early_stopping_cb, model_checkpoint_cb, tensorboard_cb]

In [17]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Nadam(learning_rate=5e-5),
              metrics=["accuracy"])

In [18]:
model.fit(
    X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), callbacks=[callbacks]
)

Epoch 1/100
1250/1250 [==============================] - 16s 11ms/step - loss: 2.3033 - accuracy: 0.1016 - val_loss: 2.3036 - val_accuracy: 0.0952
Epoch 2/100
 552/1250 [============>.................] - ETA: 7s - loss: 2.3032 - accuracy: 0.1014

KeyboardInterrupt: 

In [ ]:
model.evaluate(X_test, y_test)